In [1]:
# -*- coding: utf-8 -*-

# This code is part of Qiskit.
#
# (C) Copyright IBM 2017, 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
"""Transmon Pocket.

.. code-block::
     ________________________________
    |______ ____           __________|
    |      |____|         |____|     |
    |        __________________      |
    |       |                  |     |
    |       |__________________|     |
    |                 |              |
    |                 x              |
    |        _________|________      |
    |       |                  |     |
    |       |__________________|     |
    |        ______                  |
    |_______|______|                 |
    |________________________________|
"""

import numpy as np
from qiskit_metal import draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit



class DoubleTransmonQubit( BaseQubit ):
    """ The DoubleTransmonQubit class
                
                Inherits `BaseQubit` class.

    Create a standard pocket Doubletransmon qubit for a ground plane,
    with two transmon very close to each other having a Josephson junction in between their transmon pads (see drawing above).

    Connector lines can be added using the `connection_pads`
    dictionary. Each connector pad has a name and a list of default
    properties.

    .. image::
        Double_transmon_pocket.png

    .. meta::
        DoubleTransmon Pocket

    BaseQubit Default Options:
        * connection_pads: Empty Dict -- The dictionary that contains all active connection lines for the qubit.
        * _default_connection_pads: Empty Dict -- The default values for the (if any) connection lines of the qubit.

    Default Options:
        * pad_gap_v: '30um' -- The distance between the two charge islands in vertical transmon, which is also the resulting 'length' of the pseudo junction
        * inductor_width_v: '20um' -- Width of the pseudo junction between the two charge islands in vertical qubit  (if in doubt, make the same as pad_gap). Really just for simulating in HFSS / other EM software
        * pad_width_v: '455um' -- The width (x-axis) of the charge island pads
        * pad_height_v: '90um' -- The size (y-axis) of the charge island pads
        * pad_gap_h: '30um' -- The distance between the two charge islands in horizontal transmon, which is also the resulting 'length' of the pseudo junction
        * inductor_width_h: '20um' -- Width of the pseudo junction between the two charge islands in horizontal qubit (if in doubt, make the same as pad_gap). Really just for simulating in HFSS / other EM 
        * pad_width_h: '455um' -- The width (x-axis) of the charge island pads
        * pad_height_h: '90um' -- The size (y-axis) of the charge island pads
        * pocket_width: '650um' -- Size of the pocket (cut out in ground) along the x-axis
        * pocket_height: '650um' -- Size of the pocket (cut out in ground) along the y-axis
        * transmon_gap: '100um' --The gap in between two transmon qubits 
        * _default_connection_pads: Dict
            * pad_gap_v: '15um' -- Space between the connector pad and the charge island it is nearest to
            * pad_width_v: '125um' -- Width (x-axis) of the connector pad
            * pad_height_v: '30um' -- Height (y-axis) of the connector pad
            * pad_gap_h: '15um' -- Space between the connector pad and the charge island it is nearest to
            * pad_width_h: '125um' -- Width (x-axis) of the connector pad
            * pad_height_h: '30um' -- Height (y-axis) of the connector pad
            * pad_cpw_shift: '5um' -- Shift the connector pad cpw line by this much away from qubit
            * pad_cpw_extent: '25um' -- Shift the connector pad cpw line by this much away from qubit
            * cpw_width: 'cpw_width' -- Center trace width of the CPW line
            * cpw_gap: 'cpw_gap' -- Dielectric gap width of the CPW line
            * cpw_extend: '100um' -- Depth the connector line extense into ground (past the pocket edge)
            * pocket_extent: '5um' -- How deep into the pocket should we penetrate with the cpw connector (into the fround plane)
            * pocket_rise: '65um' -- How far up or downrelative to the center of the transmon should we elevate the cpw connection point on the ground plane
            * loc_W: '+1' -- Width location  only +-1
            * loc_H: '+1' -- Height location only +-1
    """
    default_options = Dict(
        pad_gap_v='30um',
        inductor_width_v='20um',
        pad_width_v='90um',
        pad_height_v='455um',
        pad_gap_h='30um',
        inductor_width_h='20um',
        pad_width_h='455um',
        pad_height_h='90um',
        pocket_width='400um',
        pocket_height='1000um',
        transmon_gap = '100um')
        # 90 has dipole aligned along the +X axis,
        # while 0 has dipole aligned along the +Y axis
    _default_connection_pads=Dict(
            pad_gap_v='15um',
            pad_width_v='90um',
            pad_height_v='125um',
            pad_gap_h='15um',
            pad_width_h='90um',
            pad_height_h='125um',
            pad_cpw_shift='5um',
            pad_cpw_extent='25um',
            cpw_width='cpw_width',
            cpw_gap='cpw_gap',
            # : cpw_extend: how far into the ground to extend the CPW line from the coupling pads
            cpw_extend='100um',
            pocket_extent='5um',
            pocket_rise='65um',
            loc_W='+1',  # width location  only +-1
            loc_H='+1',  # height location only +-1
            )
    """Default drawing options"""

    component_metadata = Dict(short_name='Pocket',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""

    TOOLTIP = """The base `DoubleTransmonPocket` class."""
    def make(self):
        """Define the way the options are turned into QGeometry.

        The make function implements the logic that creates the geoemtry
        (poly, path, etc.) from the qcomponent.options dictionary of
        parameters, and the adds them to the design, using
        qcomponent.add_qgeometry(...), adding in extra needed
        information, such as layer, subtract, etc.
        """
        self.make_pocket()
        self.make_connection_pads()

    def make_pocket(self):
        """Makes standard transmon in a pocket."""

        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p

        # extract chip name
        chip = p.chip

        # since we will reuse these options, parse them once and define them as varaibles
        pad_width_v= p.pad_width_v
        pad_height_v = p.pad_height_v
        pad_gap_v = p.pad_gap_v
        pad_width_h= p.pad_width_h
        pad_height_h = p.pad_height_h
        pad_gap_h = p.pad_gap_h
        transmon_gap = p.transmon_gap

        # make the pads as rectangles (shapely polygons)
        pad= draw.rectangle(pad_width_v, pad_height_v)
        pad_top_v = draw.translate(pad, 0, +(pad_height_v + pad_gap_v) / 2.)
        pad_bot_v = draw.translate(pad, 0, -(pad_height_v + pad_gap_v) / 2.)
        pad_left_h = draw.translate(pad, +((pad_width_v + pad_width_h)/2. + transmon_gap), -(pad_height_h + pad_gap_h) / 2.)
        pad_right_h = draw.translate(pad, +((pad_width_v + pad_width_h)/2. + transmon_gap + pad_gap_h + pad_width_h), -(pad_height_h + pad_gap_h) / 2.)

        rect_jj_v= draw.LineString([(0, -pad_gap_v / 2), (0, +pad_gap_v/ 2)])
        rect_jj_h= draw.LineString([(+(pad_width_v/2. + transmon_gap + pad_width_h), -(pad_gap_v+pad_height_h) / 2), (+(pad_width_v/2. + transmon_gap + pad_width_h + pad_gap_h), -(pad_gap_v+pad_height_h) / 2)])
        # the draw.rectangle representing the josephson junction
        # rect_jj = draw.rectangle(p.inductor_width, pad_gap)

        rect_pk = draw.rectangle(p.pocket_width, p.pocket_height)

        # Rotate and translate all qgeometry as needed.
        # Done with utility functions in Metal 'draw_utility' for easy rotation/translation
        # NOTE: Should modify so rotate/translate accepts qgeometry, would allow for
        # smoother implementation.
        polys = [rect_jj_v,rect_jj_h, pad_top_v, pad_bot_v,pad_left_h, pad_right_h,rect_pk]
        polys = draw.rotate(polys, p.orientation, origin=(0, 0))
        polys = draw.translate(polys, p.pos_x, p.pos_y)
        [rect_jj_v, rect_jj_h, pad_top_v, pad_bot_v, pad_left_h, pad_right_h, rect_pk]= polys

        # Use the geometry to create Metal qgeometry
        self.add_qgeometry('poly',
                           dict(pad_top_v=pad_top_v, pad_bot_v=pad_bot_v),
                           chip=chip)
        self.add_qgeometry('poly',
                           dict(pad_left_h=pad_left_h, pad_right_h=pad_right_h),
                           chip=chip)
        self.add_qgeometry('poly',
                           dict(rect_pk=rect_pk),
                           subtract=True,
                           chip=chip)
        # self.add_qgeometry('poly', dict(
        #     rect_jj=rect_jj), helper=True)
        self.add_qgeometry('junction',
                           dict(rect_jj_v=rect_jj_v),
                           width=p.inductor_width_v,
                           chip=chip)
        self.add_qgeometry('junction',
                           dict(rect_jj_h=rect_jj_h),
                           width=p.inductor_width_h,
                           chip=chip)

    def make_connection_pads(self):
        """Makes standard transmon in a pocket."""
        for name in self.options.connection_pads:
            self.make_connection_pad(name)

    def make_connection_pad(self, name: str):
        """Makes n individual connector.

        Args:
            name (str) : Name of the connector
        """

        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p
        pc = self.p.connection_pads[name]  # parser on connector options

        # extract chip name
        chip = p.chip

        # define commonly used variables once
        cpw_width = pc.cpw_width
        cpw_extend = pc.cpw_extend
        pad_width_v = pc.pad_width
        pad_height_v = pc.pad_height
        pad_cpw_shift = pc.pad_cpw_shift
        pocket_rise = pc.pocket_rise
        pocket_extent = pc.pocket_extent

        # Define the geometry
        # Connector pad
        connector_pad_v = draw.rectangle(pad_width_v, pad_height_v, -pad_width_v/ 2,
                                       pad_height_v / 2)
        
        # Connector CPW wire
        connector_wire_path = draw.wkt.loads(f"""LINESTRING (\
            0 {pad_cpw_shift+cpw_width/2}, \
            {pc.pad_cpw_extent}                           {pad_cpw_shift+cpw_width/2}, \
            {(p.pocket_width-p.pad_width)/2-pocket_extent} {pad_cpw_shift+cpw_width/2+pocket_rise}, \
            {(p.pocket_width-p.pad_width)/2+cpw_extend}    {pad_cpw_shift+cpw_width/2+pocket_rise}\
                                        )""")
        # for connector cludge
        connector_wire_CON = draw.buffer(connector_wire_path, cpw_width /
                                         2.)  # helper for the moment

        # Position the connector, rot and tranlate
        loc_W, loc_H = float(pc.loc_W), float(pc.loc_H)
        if float(loc_W) not in [-1., +1.] or float(loc_H) not in [-1., +1.]:
            self.logger.info(
                'Warning: Did you mean to define a transmon qubit with loc_W and'
                ' loc_H that are not +1 or -1?? Are you sure you want to do this?'
            )
        objects = [connector_pad, connector_wire_path, connector_wire_CON]
        objects = draw.scale(objects, loc_W, loc_H, origin=(0, 0))
        objects = draw.translate(
            objects,
            loc_W * (p.pad_width) / 2.,
            loc_H * (p.pad_height + p.pad_gap / 2 + pc.pad_gap))
        objects = draw.rotate_position(objects, p.orientation,
                                       [p.pos_x, p.pos_y])
        [connector_pad, connector_wire_path, connector_wire_CON] = objects

        self.add_qgeometry('poly', {f'{name}_connector_pad': connector_pad},
                           chip=chip)
        self.add_qgeometry('path', {f'{name}_wire': connector_wire_path},
                           width=cpw_width,
                           chip=chip)
        self.add_qgeometry('path', {f'{name}_wire_sub': connector_wire_path},
                           width=cpw_width + 2 * pc.cpw_gap,
                           subtract=True,
                           chip=chip)

        ############################################################

        # add pins
        points = np.array(connector_wire_path.coords)
        self.add_pin(name,
                     points=points[-2:],
                     width=cpw_width,
                     input_as_norm=True,
                     chip=chip)

C:\Users\uday mathur\anaconda3\envs\Qiskit_metal\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.12.0-CAPI-1.18.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\users\uday mathur\qiskit-metal\qiskit_metal\__init__.py:30: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapel

In [13]:
from qiskit_metal import designs, MetalGUI

In [14]:
design = designs.DesignPlanar()
gui = MetalGUI( design )
design.overwrite_enabled = True

In [15]:
options = Dict( pocket_width = '600um', pocket_height = '1000um', pad_width_v = '100um', pad_height_v = '200um', pad_width_h = '100um', pad_height_h = '200um', transmon_gap = '5um')

q1 = DoubleTransmonQubit( design, 'q1', options = options)
gui.rebuild()
gui.autoscale()